In [ ]:
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset,DataLoader

In [ ]:
df = pd.read_csv('/content/Clean_Dataset.csv')

In [ ]:
df.head()

,Unnamed: 0,airline,flight,source_city,departure_time,stops,arrival_time,destination_city,class,duration,days_left,price
0,0,SpiceJet,SG-8709,Delhi,Evening,zero,Night,Mumbai,Economy,2.17,1,5953
1,1,SpiceJet,SG-8157,Delhi,Early_Morning,zero,Morning,Mumbai,Economy,2.33,1,5953
2,2,AirAsia,I5-764,Delhi,Early_Morning,zero,Early_Morning,Mumbai,Economy,2.17,1,5956
3,3,Vistara,UK-995,Delhi,Morning,zero,Afternoon,Mumbai,Economy,2.25,1,5955
4,4,Vistara,UK-963,Delhi,Morning,zero,Morning,Mumbai,Economy,2.33,1,5955


In [ ]:
df.drop(['Unnamed: 0','flight'],axis =1,inplace= True)

In [ ]:
df

,airline,source_city,departure_time,stops,arrival_time,destination_city,class,duration,days_left,price
0,SpiceJet,Delhi,Evening,zero,Night,Mumbai,Economy,2.17,1,5953
1,SpiceJet,Delhi,Early_Morning,zero,Morning,Mumbai,Economy,2.33,1,5953
2,AirAsia,Delhi,Early_Morning,zero,Early_Morning,Mumbai,Economy,2.17,1,5956
3,Vistara,Delhi,Morning,zero,Afternoon,Mumbai,Economy,2.25,1,5955
4,Vistara,Delhi,Morning,zero,Morning,Mumbai,Economy,2.33,1,5955
...,...,...,...,...,...,...,...,...,...,...
300148,Vistara,Chennai,Morning,one,Evening,Hyderabad,Business,10.08,49,69265
300149,Vistara,Chennai,Afternoon,one,Night,Hyderabad,Business,10.42,49,77105
300150,Vistara,Chennai,Early_Morning,one,Night,Hyderabad,Business,13.83,49,79099
300151,Vistara,Chennai,Early_Morning,one,Evening,Hyderabad,Business,10.00,49,81585


In [ ]:
df.drop(['days_left'],axis =1,inplace= True)


In [ ]:
df.sample(4)

,airline,source_city,departure_time,stops,arrival_time,destination_city,class,duration,price
218814,Air_India,Delhi,Afternoon,one,Afternoon,Kolkata,Business,24.00,57405
13347,Air_India,Delhi,Early_Morning,one,Night,Bangalore,Economy,16.00,17812
72835,Indigo,Mumbai,Early_Morning,zero,Early_Morning,Hyderabad,Economy,1.33,2901
203794,Air_India,Chennai,Evening,one,Evening,Hyderabad,Economy,24.58,13805


In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
encoder = LabelEncoder()

In [ ]:
for x in df.columns[:7]:
  df[x] = encoder.fit_transform(df[x])

In [ ]:
df.sample(5)

,airline,source_city,departure_time,stops,arrival_time,destination_city,class,duration,price
167250,5,3,5,0,2,0,1,20.17,5441
287460,5,3,4,0,5,1,0,12.67,51707
232037,1,5,0,0,4,0,0,16.92,54608
6627,2,2,0,2,2,5,1,2.25,2410
262700,5,4,4,0,5,2,0,13.33,92193


In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
x_train,x_test,y_train,y_test = train_test_split(df.iloc[:,0:8].values,df.iloc[:,-1].values,test_size=0.2,random_state=42)

In [ ]:
y_test

array([ 7366, 64831,  6195, ...,  6734,  5082, 66465])

In [ ]:
class Data(Dataset):
  def __init__(self,x_train,y_train):
    super().__init__()
    self.x = torch.from_numpy(x_train).type(torch.float32)
    self.y = torch.from_numpy(y_train).type(torch.float32)

  def __len__(self):
    return len(self.x)
  def __getitem__(self,id):
    return self.x[id],self.y[id]

In [ ]:
train_data = Data(x_train,y_train)


In [ ]:
train_loader = DataLoader(train_data, batch_size = 10,shuffle=True)

In [ ]:
class Regression(nn.Module):
  def __init__(self):
    super().__init__()
    self.stack = nn.Sequential(
        torch.nn.Linear(8,32),
        torch.nn.ReLU(),
        torch.nn.Linear(32,16),
        torch.nn.ReLU(),
        torch.nn.Linear(16,1),
    )
  def forward(self,x:torch.tensor):
    return self.stack(x)


In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [ ]:
model = Regression()
loss_fn = nn.L1Loss()
optim = torch.optim.Adam(model.parameters(),lr=0.01)

epochs = 40

for i in range(epochs):
  temp = []
  for j,(features,targets) in enumerate(train_loader):
    model.train()
    y_pred = model(features)
    loss = loss_fn(y_pred,targets)
    temp.append(loss)
    optim.zero_grad()
    loss.backward()
    optim.step()
  model.eval()
  print(f"Epoch : {i+1} | MAE : {torch.mean(torch.tensor(temp))}")


Epoch : 1 | MAE : 16184.4716796875
Epoch : 2 | MAE : 16138.2109375
Epoch : 3 | MAE : 16137.43359375
Epoch : 4 | MAE : 16134.8125
Epoch : 5 | MAE : 16135.607421875
Epoch : 6 | MAE : 16132.0322265625
Epoch : 7 | MAE : 16132.248046875
Epoch : 8 | MAE : 16130.3935546875
Epoch : 9 | MAE : 16129.3896484375
Epoch : 10 | MAE : 16129.7607421875
Epoch : 11 | MAE : 16129.509765625
Epoch : 12 | MAE : 16129.23046875
Epoch : 13 | MAE : 16127.6533203125
Epoch : 14 | MAE : 16129.20703125
Epoch : 15 | MAE : 16128.8359375
Epoch : 16 | MAE : 16124.5390625
Epoch : 17 | MAE : 16126.5634765625
Epoch : 18 | MAE : 16126.4619140625
Epoch : 19 | MAE : 16127.642578125
Epoch : 20 | MAE : 16127.2333984375
Epoch : 21 | MAE : 16127.6240234375
Epoch : 22 | MAE : 16125.22265625
Epoch : 23 | MAE : 16125.3349609375
